# 05 How to Plot Winddata

This chapter is all about plotting wind data. First we will deal with 10m winds in x- and y-direction and second we will have a look at 4D-data: wind at different heights!

The very first, we import our modules and have some presettings again:

In [ ]:
# Display the plots in the notebook:
%matplotlib inline

In [ ]:
# Import the tools we are going to need today:
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # netCDF library
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size
np.set_printoptions(threshold=20)  # avoid to print very large arrays on screen
# The commands below are to ignore certain warnings.
import warnings
warnings.filterwarnings('ignore')

## 05_01 3D-Data: Surface Winds

Get the data:

In [ ]:
ds = xr.open_dataset('./data/ERA5_LowRes_Monthly_uvslp_Fabien.nc')
print(ds)

Let's select a year out of our data and print our variable:

In [ ]:
u = ds.u10.sel(time = '2018')
u

On the second line, you should see something similar to:

    [1156800 values with dtype=float32]
    
As we are going to see, this is a "trick" that xarray uses to hide an important information from us: xarray didn't actually read the data! However, as we have seen, xarray is very capable to plot this data or do mathematical operations on them. 

So why didn't xarray read the data yet? 

Because it *knows* that we don't need it right away. This feature is called [lazy evaluation](https://en.wikipedia.org/wiki/Lazy_evaluation) in computer sciences. It is extremely useful, as it spares time and memory by loading the data only when really needed.

In some cases this feature can be undesirable. For example: when reading the data is quite slow. In theses exercises, we are working with data on remote directories, and the data we are using is highly compressed (the size that the data takes on disk is *very* small in comparison to the space it takes in memory). Reading compressed data takes time, and the operations we are going to make below make decompression even slowier.

This is why I'll sometimes recommend to do the following:

In [ ]:
u = ds.u10.sel(time = '2019').load()
u

Now we can actually see the data in the form of an array!

We are now calculating the **averaged u and v windfields at the 10 m level for the month of January**:

In [ ]:
u = ds.u10.groupby('time.month').mean(dim = 'time').sel(month=1).load()
v = ds.v10.groupby('time.month').mean(dim = 'time').sel(month=1).load()

### 05_01_01 Quiver Plots

One traditional way to plot the wind data is as vector arrows. We can use a function called [quiver(X, Y, U, V)](http://matplotlib.org/api/pyplot_api.html?highlight=quiver#matplotlib.pyplot.quiver) for that. 
It takes the following arguments: U and V are the arrow data, X and Y set the location of the arrows. You can also add a C argument, that will set the color of the arrows. 

For example:

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())  # note that I changed the map projection
qv = ax.quiver(u.longitude, u.latitude, u, v, transform=ccrs.PlateCarree())
ax.coastlines(color='grey');

Well, this does not really look useful. The problem is, that there are too many grid points for the plot to be able to illustrate the data as single arrows. So the solution is, to not use every grid point but e.g. only every 10th!

Let's try it:

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())  
pu, pv = u[::9,::9], v[::9,::9]  # we will discuss what this line does in the cells below
qv = ax.quiver(pu.longitude, pu.latitude, pu, pv, transform=ccrs.PlateCarree())
ax.coastlines(color='grey');
pu

Note the new syntax in the line ``pu, pv = u[::10,::10], v[::10,::10]``. Let's explain it in detail. the ``::9`` is the Numpy way to index each 9th element of an array. It is called [slicing](http://docs.scipy.org/doc/numpy-1.10.0/reference/arrays.indexing.html#basic-slicing-and-indexing) in the numpy jargon. We already learned how to slice with a slice object!

The `start:stop:step` syntax is actually creating such a python slice object too. So it is just a more convenient way, to do slicing in python.
Above, `start` and `stop` were not defined. That means: take the whole array, from the start until the end!


Let's try this new form of slicing on a simpler array:

In [ ]:
a = np.arange(11)
print(a)

Underneath there will be some different slicings, each explained right next to the command.

In [ ]:
a[:] #takes all elements of the array

In [ ]:
a[2:] #takes the elements from the second one until the end. Neither the stop nor the step value are defined, 
      # so python will assume you want the data until the end of the array in steps of one (default)

In [ ]:
a[:-1] #takes all the elements from the start until the second last one: -1 means the end of the array -1

In [ ]:
a[2:5] #start from the third element (remember: in python the firs element is number 0!) and take every element until 
       #the fourth (remember: in python the stop index is excluded!)

In [ ]:
a[::-1] #take all elements of the array, the step value is minus one which means: step trough them backwards!
        # --> we get the reversed array

In [ ]:
a[::-2] #same as above but only take every second element. --> reversed array with every second element

OK, so that was slincing in one dimensions. Slicing also works in N-dimensions. For example:

In [ ]:
a = np.arange(4*5).reshape((4, 5)) #don't worry about how we created this array, it does not matter for us now
print(a)

In [ ]:
a[:,::-1] # the first slice corresponds to the columns, the second one to the rows!

In [ ]:
a[::2,::2]

OK, that was slicing in two dimensions, so we now better understand what ``u[::9,::9]`` means. But what was about that comma in between? This is simply what people call "syntaxic sugar", a nice and easy way to write one line in two: 

In [ ]:
a, b = 12, 'Hello'
print(a)
print(b)

This kind of shortcuts should be used only if:
- the lines of code are easy
- the two variables are somehow related (for example u and v winds)

We can now try some different slicing for our quiver plot:

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())  
pu, pv = u[::5,::5], v[::5,::5]
qv = ax.quiver(pu.longitude, pu.latitude, pu, pv, transform=ccrs.PlateCarree())
ax.coastlines(color='grey');

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())  
pu, pv = u[::15,::15], v[::15,::15]
qv = ax.quiver(pu.longitude, pu.latitude, pu, pv, transform=ccrs.PlateCarree())
ax.coastlines(color='grey');

Or change the color of our arrows:

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())  
pu, pv = u[::10,::10], v[::10,::10]
qv = ax.quiver(pu.longitude, pu.latitude, pu, pv, color = 'r', transform=ccrs.PlateCarree())
ax.coastlines(color='grey');

It seems reasonable, to somehow better visualize the windspeed than just by the length of the arrows. We can for example plot the wind speed as a shaded color plot (as we did for temperature and precipitation) and plot the wind arrows on top of it.! 

Therefore, we must first determine the windspeed:

In [ ]:
ws = (u**2 + v**2)**0.5

Now plot the windspeed first and on top of it the arrows:

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())  
ws.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), cmap='cool', levels=np.arange(0, 15), cbar_kwargs={'label':'Windspeed [m/s]'});
pu, pv = u[::10,::10], v[::10,::10]
qv = ax.quiver(pu.longitude, pu.latitude, pu, pv, transform=ccrs.PlateCarree())
ax.coastlines();
ax.set_title('Average Wind and Windspeed January');

### 05_01_02 Streamline Plots

Another way to plot wind data are the so called [streamlines](http://matplotlib.org/api/pyplot_api.html?highlight=streamplot#matplotlib.pyplot.streamplot):

In [ ]:
fig = plt.figure('figsize', [12,5])
ax = plt.axes(projection=ccrs.PlateCarree())
ax.streamplot(u.longitude, u.latitude, u.values, v.values, transform=ccrs.PlateCarree(), density=4)
ax.coastlines(color='grey');

The `u.values` syntax extracts the actual u values as a Numpy NdArray from the XArray DataArray! That is what we need as an argument for the streamplot function!

In [ ]:
u.values

In [ ]:
type(u.values)

**Note**: if the above plot was too slow for your taste, you can slice your data like we did for the quiver plot, this will produce the plot faster. Anyway, this does not have any impact on the denseness of the streamlines!

For example:

In [ ]:
pu, pv = u[::10,::10], v[::10,::10]
ax = plt.axes(projection=ccrs.PlateCarree())
ax.streamplot(pu.longitude, pu.latitude, pu.values, pv.values, transform=ccrs.PlateCarree(), density=4)
ax.coastlines(color='grey');

By changing the `density` argument, you can control the closeness of streamlines. The deault value is density = 1.
Let's try it:

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.streamplot(u.longitude, u.latitude, u.values, v.values, transform=ccrs.PlateCarree(), density=2)
ax.coastlines(color='grey');

You can also vary the linewidth of the streamlines by specifying the argument `linewidth`. You can either set it to a numeric or a 2D-array. 

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
strm = ax.streamplot(u.longitude, u.latitude, u.values, v.values, transform=ccrs.PlateCarree(),
                     density=2, linewidth = 3)
ax.coastlines(color='grey'); ax.gridlines(draw_labels=True);

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
strm = ax.streamplot(u.longitude, u.latitude, u.values, v.values, transform=ccrs.PlateCarree(),
                     density=5, linewidth = 0.5)
ax.coastlines(color='grey'); ax.gridlines(draw_labels=True);

A problem with streamlines is that they provide no information about the strength of the flow. Again, it is possible to display this information with colors. Just define the `color` argument as the windspeed array `ws` we already created further up! In order to make this work, we again have to use `ws.values`, since color wants a Numpy Array and not a DataArray as input.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
strm = ax.streamplot(u.longitude, u.latitude, u.values, v.values, transform=ccrs.PlateCarree(),
                     density=4, color=ws.values, cmap='cool')
bar = plt.colorbar(strm.lines) # add a colorbar defined by the lines of the streamplot (strm.lines)
bar.set_label('Windspeed [m/s]') #give a label to the colorbar
ax.coastlines(color='grey'); ax.gridlines(draw_labels=True);

### 05_01_03 1D-Plots

We can also display wind data in a 1D-Plot rather than in a 2D one as the above examples. A possible and useful form of representation would be the zonal average of the meridional and the zonal 10m winds:

In [ ]:
ds.u10.mean(dim=['time', 'longitude']).plot();
plt.axhline(0, color='k');
plt.xlim([-90, 90]);
plt.title('Zonal Wind at 10m');
plt.ylabel('m s$^{-1}$');
plt.xlabel('° North');
plt.figure();
ds.v10.mean(dim=['time', 'longitude']).plot();
plt.axhline(0, color='k');
plt.xlim([-90, 90]);
plt.title('Meridional Wind at 10m');
plt.ylabel('m s$^-1$');
plt.xlabel('° North');

We encounter a new syntax in the above commands: `plt.ylabel('m s$^{-1}$')`. What do those dollar signs mean? 

In matplotlib, one can write [**mathematical expressions**](https://matplotlib.org/3.1.1/tutorials/text/mathtext.html) by using dollar signs as a wrapper of the expressions. Everything that is written in between of two $-signs is recognised as a mathematical expression by matplotlib.

* To make subscripts and superscripts, you can use the `_` and `^` symbols. In the above example, we used `^` to create a superscript of -1. 
* Radicals can be produced with the `\sqrt[]{}` command. For example: `\sqrt{2}`. Any base can optionally be provided inside the square brackets. 
* Look at the webpage above, to get to know more. E.g. how to write mathematical symbols or greek letters.

As another example, we can plot the monthly means of the windspeed for the year 2018 at the location of Innsbruck:

In [ ]:
u10_ibk_18 = ds.u10.sel(latitude = 47.25, longitude = 11.25, time = '2018') #select Innsbruck and year 2018
v10_ibk_18 = ds.v10.sel(latitude = 47.25, longitude = 11.25, time = '2018') #select Innsbruck and year 2018
ws_ibk_18 = ws = (u10_ibk_18**2 + v10_ibk_18**2)**0.5 #compute windspeed for Innsbruck 2018

ax = ws_ibk_18.plot();
plt.title('Windspeed 2018 @ 47.25°North, 11.25°East ');
plt.ylabel('m s$^{-1}$');
plt.xlabel('');
plt.xlim(['2018-01','2018-12']);

## 05_02 4D-Data: Wind at different pressure levels

We will now deal with a new kind of dataset: 4D-Data. Let's open such a dataset and see what it looks like:

In [ ]:
ds = xr.open_dataset('./data/ERA5_LowRes_MonthlyAvg_4D_zuvw_Fabien.nc')
ds

We have a new coordinate axis: level! Those are the different pressure levels, at which wind data (u,v,w) and height data (z) is available.

In [ ]:
ds.level

The data is available at the pressure levels 50, 100, 200, 300, 400, 500, 700, 850 and 1000 millibars. Millibars are equivalent to hectopascal. 

We can use this new data, to plot the height- (= pressure-) dependency of the horizontal as well as the vertical windfields! Before we do this, let's have a quick look at our variables: u, v, w, and z.

In [ ]:
ds.u

In [ ]:
ds.z # this is the geopotential! get the geopotential height by dividing by g

We notice furthermore, that the time coordinate changed to a **month coordinate**! The data is already grouped by months and averaged.

### 05_02_01 Vertical Veloctiy

In [ ]:
ds.w

Let's select southamerica @500 hPa, average over time and plot the vertical veloctiy w:

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
w = ds.w.mean(dim='month').sel(level=500, longitude = slice(-80,-20), latitude = slice(20,-60))
w.plot(transform = ccrs.PlateCarree() )
ax.set_title('500 hPa Mean Vertical Winds')
ax.coastlines(color='grey'); ax.gridlines();

Sinking air at the upstream side of the andes and rising air at the downstream side? This does not seem very reasonable. We need to change the sign of w!


In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
w = ds.w.mean(dim='month').sel(level=500, longitude = slice(-80,-20), latitude = slice(20,-60))*-1
w.plot(transform = ccrs.PlateCarree() )
ax.set_title('500 hPa Mean Vertical Winds')
ax.coastlines(color='grey'); ax.gridlines();

Better.

We can now plot the vertical windfield in a different way. One option would be, to take a certain area, average the zonal winds over that area and plot them as a function of the month!

E.g. for the African Horn:

In [ ]:
w_horn=ds.w.sel(longitude=slice(40, 52), latitude=slice(15, -5)).mean(dim= ['longitude', 'latitude'])
w_horn *= -1
plt.figure(figsize=(20,7))
w_horn.T.plot.contourf(levels=30, cbar_kwargs={'label':'m s$^{-1}$'})
plt.ylim([1000,50])
plt.xlim([1,12])
plt.yscale('log')
plt.title('Vertical Wind distibution over the North- and South-Eastern Horn (40° to 55°E and 5°S to 15°N)');

Another option: average over longitude and time and plot the vertical wind as a function of latitude!

In [ ]:
w_horn=ds.w.mean(dim= ['longitude', 'month']).sel(latitude = slice(30,-30))
w_horn *= -1
plt.figure(figsize=(20,7))
w_horn.plot.contourf(levels=30, cbar_kwargs={'label':'m s$^{-1}$'})
plt.ylim([1000,50])
plt.ylabel('Pressure [hPa]')
plt.xlabel('Latitude [°N]')
plt.yscale('log')
plt.title('Mean Vertical Wind Distribution from -30 to 30°N');

### 05_02_02 Zonal Average Plots

It's pretty easy, to produce a zonal plot of the vertical structure of the atmosphere:

In [ ]:
u_allz = ds.u.sel(month = 1).load() #select average of january
u_allz = u_allz.mean(dim='longitude')
u_allz

In [ ]:
u_allz.plot();

We notice, that the y-axis needs to be reversed in order to go from the ground up to the higher atmosphere! Furthermore, the imshow plot does not really look nice. Let's make a contourf plot instead:

In [ ]:
u_allz.plot.contourf(levels = np.arange(-45, 45, 5),  cbar_kwargs={'label':'U-wind (m s$^{-1}$)'});
plt.ylim([1000, 50]); #reverse the y-axis
plt.xlim([-90,90]);
plt.ylabel('Pressure [hPa]');
plt.title('Average Zonal Winds January');

To represent more acurately the real altitude of the winds, a logarithmic scale might be better:

In [ ]:
u_allz.plot.contourf(levels = np.arange(-45, 45, 5),  cbar_kwargs={'label':'U-wind (m s$^{-1}$)'});
plt.ylim([50,1000]);
plt.xlim([-90,90]);
plt.ylabel('Pressure [hPa]');
plt.title('Average Zonal Winds January')
plt.yscale('log') #change yscale from linear to logarithmic!

One can also take a certain area, average the zonal winds over that area and plot them as a function of the month!

We try this for the African Horn:

In [ ]:
u_horn=ds.u.sel(longitude=slice(50,70), latitude=slice(12, -3)).mean(dim= ['longitude', 'latitude'])
plt.figure(figsize=(20,7))
u_horn.T.plot.contourf(levels=30, cbar_kwargs={'label':'m s$^{-1}$'}, cmap = 'PuOr')
plt.ylim([1000,50])
plt.yscale('log')
plt.title('Zonal Wind distibution over the North- and South-Eastern Horn (40° to 55°E and 5°S to 15°N)');